<a href="https://colab.research.google.com/github/Tasfiq23/Keras_DNN/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-09-16 16:34:12--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-16 16:34:12 (63.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [3]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2021-09-16 16:34:18--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.121.128, 108.177.120.128, 142.250.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.121.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2021-09-16 16:34:18 (85.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df['target']

0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64

In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"Target{target}" , "(real disaster)" if target > 0 else "(not a real disaster)" )
  


Target0 (not a real disaster)
Target0 (not a real disaster)
Target0 (not a real disaster)
Target1 (real disaster)
Target0 (not a real disaster)


In [8]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [9]:
train_sentences[3:]

array(["@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [10]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    ngrams = None,
                                    output_mode='int',
                                    output_sequence_length=None)

In [11]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [12]:
max_vocab_length= 10000 #max number of words in vocubulary
max_length =15 #max length of our sequence to be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    ngrams = None,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [13]:
text_vectorizer.adapt(train_sentences)

In [14]:
sample_sentence = ' There is a flood in my street'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [15]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding 

In [16]:
embedding= Embedding(input_dim=max_vocab_length,
                     output_dim=128,
                     input_length=max_length)

embedding

In [17]:
sample_embeded = embedding(text_vectorizer([sample_sentence]))

In [18]:
sample_embeded[0]

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[-0.01076096,  0.04311944, -0.0460475 , ...,  0.04541809,
         0.00517521,  0.00084624],
       [-0.04923379, -0.02580727, -0.04965037, ..., -0.00077533,
        -0.04081222,  0.0179776 ],
       [ 0.00337107, -0.04683385, -0.01308846, ...,  0.0291649 ,
         0.02952475,  0.03047388],
       ...,
       [ 0.02672011,  0.02111179, -0.04188286, ..., -0.02629058,
        -0.01092737, -0.04384453],
       [ 0.02672011,  0.02111179, -0.04188286, ..., -0.02629058,
        -0.01092737, -0.04384453],
       [ 0.02672011,  0.02111179, -0.04188286, ..., -0.02629058,
        -0.01092737, -0.04384453]], dtype=float32)>

#Model 0: Getting a baseline

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [20]:
#create tokenization and modeling pipeline

model_0 = Pipeline([
                    ("tfidf",TfidfVectorizer()),
                     ('clf',MultinomialNB())
])

#Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [21]:
baseline_score= model_0.score(val_sentences,val_labels)

In [22]:
baseline_score

0.7926509186351706

In [23]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [24]:
from sklearn.metrics import accuracy_score,precision_score,f1_score,precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  """
    Calculate model accuracy score,precision score and f1 score of binary classification model

    Args:
    ----
    y_true = true labels in the form of a 1D array
    y_pred = predicted labels in the form of a 1D array

    Returns a dictionary of accuracy, precision, recall, f1-score.

    """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
    #Calculate model accuracy
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                    "precision": model_precision,
                    "recall": model_recall,
                    "f1": model_f1}
  return model_results

In [25]:
# Get baseline results
baseline_results = calculate_result(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#Model 1: Simple Dense model

In [26]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [27]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype='string')
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name='model_1_dense')

In [28]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [29]:

# Get a summary of the model
model_1.summary()


Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20210916-163426
Epoch 1/5
215/215 [==============================] - 5s 11ms/step - loss: 0.6137 - accuracy: 0.6838 - val_loss: 0.5374 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.4436 - accuracy: 0.8184 - val_loss: 0.4691 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3492 - accuracy: 0.8575 - val_loss: 0.4615 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2865 - accuracy: 0.8894 - val_loss: 0.4629 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2392 - accuracy: 0.9108 - val_loss: 0.4799 - val_accuracy: 0.7887


#model 2 LSTM

In [31]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [32]:
model_2.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

In [33]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20210916-163448
Epoch 1/5
215/215 [==============================] - 8s 16ms/step - loss: 0.2224 - accuracy: 0.9209 - val_loss: 0.5497 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1545 - accuracy: 0.9425 - val_loss: 0.5978 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1270 - accuracy: 0.9512 - val_loss: 0.6651 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1027 - accuracy: 0.9612 - val_loss: 0.7780 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0813 - accuracy: 0.9672 - val_loss: 0.9538 - val_accuracy: 0.7690


In [35]:
#Make prediction with pred probs to labels
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[2.6008908e-03],
       [6.7897797e-01],
       [9.9970067e-01],
       [3.5107538e-02],
       [5.7559152e-04],
       [9.9869484e-01],
       [8.4798574e-01],
       [9.9977368e-01],
       [9.9961156e-01],
       [3.3859354e-01]], dtype=float32)

In [36]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [37]:
model_2_results = calculate_result(y_true=val_labels,y_pred=model_2_preds)
model_2_results

{'accuracy': 76.9028871391076,
 'f1': 0.7645815397825486,
 'precision': 0.7763415602461458,
 'recall': 0.7690288713910761}

In [38]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [39]:
#build an RNN using the GRU cell

from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
print(inputs.shape)
x = text_vectorizer(inputs)
print(x.shape)
x =embedding(x)
print(x.shape)
x=layers.GRU(64,return_sequences='True')(x)
print(x.shape)
x= layers.LSTM(64,return_sequences='True')(x)
print(x.shape)
x= layers.GRU(64)(x)
print(x.shape)
x= layers.Dense(64,activation='relu')(x)
print(x.shape)
outputs = layers.Dense(1,activation='sigmoid')(x)
print(x.shape)
model_3 = tf.keras.Model(inputs,outputs)
print(x.shape)

(None, 1)
(None, 15)
(None, 15, 128)
(None, 15, 64)
(None, 15, 64)
(None, 64)
(None, 64)
(None, 64)
(None, 64)


In [40]:
model_3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 15, 64)            37248     
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 64)            33024     
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160  

In [41]:
model_3.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

In [42]:
#fit the model

model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,'model_3_GRU')]) 

Saving TensorBoard log files to: model_logs/model_3_GRU/20210916-163507
Epoch 1/5
215/215 [==============================] - 12s 30ms/step - loss: 0.1458 - accuracy: 0.9562 - val_loss: 0.8446 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0751 - accuracy: 0.9692 - val_loss: 1.0600 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0634 - accuracy: 0.9742 - val_loss: 1.1368 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 4s 21ms/step - loss: 0.0515 - accuracy: 0.9756 - val_loss: 1.3045 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0496 - accuracy: 0.9771 - val_loss: 1.6587 - val_accuracy: 0.7782


In [43]:
model_3_pred_probs =model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.8174115e-04],
       [8.7586951e-01],
       [9.9999893e-01],
       [2.4524444e-01],
       [3.7773621e-05],
       [9.9998415e-01],
       [9.9975663e-01],
       [9.9999833e-01],
       [9.9999833e-01],
       [9.8201227e-01]], dtype=float32)

In [44]:
#convert model 3 pred probs to labels

model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [45]:
model_3_results = calculate_result(y_true= val_labels ,y_pred=model_3_preds)
model_3_results

{'accuracy': 77.82152230971128,
 'f1': 0.7763875633348853,
 'precision': 0.7796479981380524,
 'recall': 0.7782152230971129}

### Bidirectional RNN

In [46]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x= text_vectorizer(inputs)
x= embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
#x= layers.Dense(64,activation='relu')(x)
outputs= layers.Dense(1,activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')


In [47]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 15, 128)           98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,453,441
Trainable params: 1,453,441
Non-trainable params: 0
___________________________________

In [48]:
model_4.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

In [49]:
#fit the model

model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210916-163540
Epoch 1/5
215/215 [==============================] - 14s 36ms/step - loss: 0.1082 - accuracy: 0.9634 - val_loss: 0.9339 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0549 - accuracy: 0.9753 - val_loss: 1.2129 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0465 - accuracy: 0.9781 - val_loss: 1.2783 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0419 - accuracy: 0.9785 - val_loss: 1.3766 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0394 - accuracy: 0.9813 - val_loss: 1.3838 - val_accuracy: 0.7612


In [50]:
model_4_pred_probs =model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[9.9749690e-01],
       [5.9901768e-01],
       [9.9996579e-01],
       [2.9796487e-02],
       [1.1548024e-04],
       [9.9994659e-01],
       [9.9481481e-01],
       [9.9997067e-01],
       [9.9996138e-01],
       [9.9898356e-01]], dtype=float32)

In [51]:
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))

In [52]:
model_4_results = calculate_result(y_true= val_labels ,y_pred=model_4_preds)
model_4_results

{'accuracy': 76.11548556430446,
 'f1': 0.761346832061735,
 'precision': 0.7617067529480163,
 'recall': 0.7611548556430446}

In [53]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,),dtype=tf.string)
x= text_vectorizer(inputs)
x= embedding(x)
x = layers.Conv1D(filters =64,kernel_size=5,activation='relu',strides=1,padding='valid')(x)
x = layers.GlobalMaxPooling1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs,outputs,name='model_5_conv1D')


model_5.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

In [54]:
model_5.summary()

Model: "model_5_conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [55]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210916-163629
Epoch 1/5
215/215 [==============================] - 26s 12ms/step - loss: 0.1298 - accuracy: 0.9577 - val_loss: 0.8620 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0781 - accuracy: 0.9714 - val_loss: 1.0081 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0646 - accuracy: 0.9752 - val_loss: 1.1272 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0573 - accuracy: 0.9768 - val_loss: 1.1733 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0522 - accuracy: 0.9800 - val_loss: 1.1862 - val_accuracy: 0.7651


In [56]:
#make prdiction
model_5_pred_probs =model_5.predict(val_sentences)
model_5_pred_probs[:10]
model_5_preds=tf.squeeze(tf.round(model_5_pred_probs))

In [57]:
model_5_results = calculate_result(y_true= val_labels ,y_pred=model_5_preds)
model_5_results

{'accuracy': 76.50918635170603,
 'f1': 0.7642522657439998,
 'precision': 0.7648238707003729,
 'recall': 0.7650918635170604}

In [59]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,"when you can the universal sentence encoder on a sentence, it turns into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01602835  0.01068851  0.02425466 -0.01405769  0.01434424  0.08292626
  0.01963372  0.0616014  -0.00352699 -0.01216414  0.00978648 -0.01248499
  0.01232344  0.09748449  0.06141113 -0.03728351  0.01860886 -0.04669854
  0.00413913 -0.06363907 -0.02469896  0.02713691  0.02284444 -0.00210024
 -0.00630593 -0.03964957  0.0222041   0.00115076 -0.03132179  0.00119526
 -0.04012549  0.04561896 -0.01530596 -0.00175918  0.02173134 -0.08450424
  0.03340028  0.04604551 -0.02480246 -0.08681665  0.00702695 -0.00770478
 -0.0143454   0.07814161 -0.10676058 -0.05152997 -0.00858157 -0.03232233
 -0.03871095  0.02581469], shape=(50,), dtype=float32)


In [61]:
from tensorflow.keras import layers

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")



In [72]:
model_6 = tf.keras.Sequential([
          sentence_encoder_layer,
          layers.Dense(64,activation='relu'),
          layers.Dense(1,activation='sigmoid')
],name="model_6_USE")
model_6.compile(optimizer='adam',
                loss= 'binary_crossentropy',
                metrics=['accuracy'])

In [73]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_9 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [74]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,'model_6_USE')])

Saving TensorBoard log files to: model_logs/model_6_USE/20210916-170050
Epoch 1/5
215/215 [==============================] - 7s 29ms/step - loss: 0.5017 - accuracy: 0.7846 - val_loss: 0.4467 - val_accuracy: 0.8005
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.4133 - accuracy: 0.8152 - val_loss: 0.4392 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 5s 21ms/step - loss: 0.4012 - accuracy: 0.8216 - val_loss: 0.4306 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3914 - accuracy: 0.8270 - val_loss: 0.4296 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.3848 - accuracy: 0.8295 - val_loss: 0.4229 - val_accuracy: 0.8176


In [75]:
#make prdiction
model_6_pred_probs =model_6.predict(val_sentences)
model_6_pred_probs[:10]
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))

In [76]:
model_6_results = calculate_result(y_true= val_labels ,y_pred=model_6_preds)
model_6_results

{'accuracy': 81.75853018372703,
 'f1': 0.8165413215854364,
 'precision': 0.8186307248506688,
 'recall': 0.8175853018372703}